# Drowsiness Detection OpenCV


This function has been set up by the learner to prevent drowsiness.

이 기능은 학습자가 졸음을 방지하기 위해 설정하였습니다.
## Applications
This feature is available to users who have studied for a long time that can lead to drowsiness.

이 기능은 졸음으로 이어질 수 있는 장시간 공부한 사용자가 사용할 수 있습니다.
### Algorithm

Each eye is represented by 6 (x, y)-coordinates, starting at the left-corner of the eye (as if you were looking at the person), and then working clockwise around the eye:.

각 눈은 눈 왼쪽 모서리에서 시작하여(사람을 보는 것처럼) 시계 방향으로 6개의 (x, y) 좌표로 표시됩니다.

In [1]:
from scipy.spatial import distance
from imutils import face_utils
import imutils
import dlib
import cv2

In [2]:
def eye_aspect_ratio(eye):
	A = distance.euclidean(eye[1], eye[5]) #eye[1]은 동공의 왼쪽 모서리와 상안검이 만나는 부분,eye[5]는 동공의 왼쪽 모서리와 하얀검이 만나는 부분이며 둘 사이의 거리를 구함
	B = distance.euclidean(eye[2], eye[4]) #eye[2]는 동공의 오른쪽 모서리와 상안검이 만나는 부분,eye[4]는 동공의 오른쪽 모서리와 하안검이 만나는 부분이며 둘 사이의 거리를 구함
	C = distance.euclidean(eye[0], eye[3]) #eye[0]은 동공은 외안각, eye[3]은 내안각이며 둘 사이의 거리를 구함
	ear = (A + B) / (2.0 * C) #눈 종횡비를 구함
	return ear #눈 종횡비를 반환

In [3]:
thresh = 0.25
frame_check = 20
detect = dlib.get_frontal_face_detector()
predict = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat") #눈감음 인식

In [4]:
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

In [5]:
cap=cv2.VideoCapture(0)
flag=0
while True:
	ret, frame=cap.read()
	frame = imutils.resize(frame, width=450)
	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
	subjects = detect(gray, 0)
	for subject in subjects:
		shape = predict(gray, subject)
		shape = face_utils.shape_to_np(shape)#converting to NumPy Array
		leftEye = shape[lStart:lEnd]
		rightEye = shape[rStart:rEnd]
		leftEAR = eye_aspect_ratio(leftEye)
		rightEAR = eye_aspect_ratio(rightEye)
		ear = (leftEAR + rightEAR) / 2.0
		leftEyeHull = cv2.convexHull(leftEye)
		rightEyeHull = cv2.convexHull(rightEye)
		cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
		cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)
		if ear < thresh:
			flag += 1
			#print (flag)
			if flag >= frame_check:
				cv2.putText(frame, "****************SLEEP!****************", (10, 30),
					cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
				cv2.putText(frame, "****************SLEEP!****************", (10,325),
					cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
		else:
			flag = 0
	cv2.imshow("Frame", frame)
	key = cv2.waitKey(1) & 0xFF
	if key == ord("q"):
		cv2.destroyAllWindows()
		cap.release()
		break
